# Benchmark for CPU vs GPU (For Machine Learning):

I decided to see the performance difference between training on a CPU vs GPU.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
# checking images shape
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
# display single image shape
X_train[0].shape

(32, 32, 3)

In [6]:
# checking labels
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [7]:
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [8]:
# one hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [9]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


In [16]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 136s 87ms/step - loss: 1.8104 - accuracy: 0.3551
Epoch 2/10
1563/1563 [==============================] - 111s 71ms/step - loss: 1.6219 - accuracy: 0.4255
Epoch 3/10
1563/1563 [==============================] - 105s 67ms/step - loss: 1.5399 - accuracy: 0.4573
Epoch 4/10
1563/1563 [==============================] - 106s 68ms/step - loss: 1.4789 - accuracy: 0.4786
Epoch 5/10
1563/1563 [==============================] - 108s 69ms/step - loss: 1.4305 - accuracy: 0.4963
Epoch 6/10
1563/1563 [==============================] - 108s 69ms/step - loss: 1.3878 - accuracy: 0.5118
Epoch 7/10
1563/1563 [==============================] - 109s 70ms/step - loss: 1.3514 - accuracy: 0.5233
Epoch 8/10
1563/1563 [==============================] - 111s 71ms/step - loss: 1.3160 - accuracy: 0.5353
Epoch 9/10
1563/1563 [==============================] - 113s 72ms/step - loss: 1.2834 - accuracy: 0.5499
Epoch 10/10
1563/1563 [==============================] 

In [10]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

2022-01-18 04:44:39.466911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 04:44:40.259465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 04:44:40.260133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 04:44:40.322717: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

Epoch 1/10
1563/1563 [==============================] - 17s 7ms/step - loss: 1.8170 - accuracy: 0.3510
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.6252 - accuracy: 0.4251
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.5421 - accuracy: 0.4563
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4837 - accuracy: 0.4775
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4322 - accuracy: 0.4957
Epoch 6/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3882 - accuracy: 0.5129
Epoch 7/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3502 - accuracy: 0.5260
Epoch 8/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3151 - accuracy: 0.5393
Epoch 9/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2839 - accuracy: 0.5499
Epoch 10/10
1563/1563 [==============================] - 11s 7ms/step - l